In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,0,Ushuaia,AR,62.26,few clouds,-54.8000,-68.3000,Albatros Hotel
1,1,Panguna,PG,69.31,overcast clouds,-6.3164,155.4848,NaN
2,4,Hilo,US,71.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
3,6,Wad Madani,SD,84.81,clear sky,14.4012,33.5199,فندق يوغسلافيا
4,8,Ambon,ID,81.57,moderate rain,-3.6954,128.1814,Swiss-Belhotel Ambon


In [11]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [12]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=3.00)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df["City"] == "Spring Valley") & (vacation_df["Hotel Name"] == "Westgate Flamingo Bay Resort")]
vacation_end = vacation_df.loc[(vacation_df["City"] == "Spring Valley") & (vacation_df["Hotel Name"] == "Westgate Flamingo Bay Resort")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Pacific Grove") & (vacation_df["Hotel Name"] == "Lovers Point Inn")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Eureka") & (vacation_df["Hotel Name"] == "Eureka Inn Trademark By Wyndham")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "North Bend") & (vacation_df["Hotel Name"] == "Red Lion Hotel Coos Bay")]  

In [26]:
vacation_stop3

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
332,508,North Bend,US,50.92,overcast clouds,43.4065,-124.2243,Red Lion Hotel Coos Bay


In [27]:
vacation_stop2

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
341,519,Eureka,US,51.37,overcast clouds,40.8021,-124.1637,Eureka Inn Trademark By Wyndham


In [28]:
vacation_stop1

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
79,123,Pacific Grove,US,56.37,clear sky,36.6177,-121.9166,Lovers Point Inn


In [29]:
vacation_start

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
250,390,Spring Valley,US,45.37,few clouds,36.108,-115.245,Westgate Flamingo Bay Resort


In [30]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start['Lat'].to_numpy()[0], vacation_start['Lng'].to_numpy()[0]
end = vacation_end['Lat'].to_numpy()[0], vacation_end['Lng'].to_numpy()[0]
stop1 = vacation_stop1['Lat'].to_numpy()[0], vacation_stop1['Lng'].to_numpy()[0]
stop2 = vacation_stop2['Lat'].to_numpy()[0], vacation_stop2['Lng'].to_numpy()[0]
stop3 = vacation_stop3['Lat'].to_numpy()[0], vacation_stop3['Lng'].to_numpy()[0]
print(f'Starting Vacation at: {start}./n Then make your first stop at: {stop1}./n Followed by stop 2: {stop2}./n Proceed to stop 3: {stop3}./n Ending at: {end}.')

Starting Vacation at: (36.108, -115.245)./n Then make your first stop at: (36.6177, -121.9166)./n Followed by stop 2: (40.8021, -124.1637)./n Proceed to stop 3: (43.4065, -124.2243)./n Ending at: (36.108, -115.245).


In [32]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(
    start, end, waypoints=[stop1, stop2, stop3],
    travel_mode= 'DRIVING')

fig.add_layer(vacation_itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,390,Spring Valley,US,45.37,few clouds,36.1080,-115.2450,Westgate Flamingo Bay Resort
1,123,Pacific Grove,US,56.37,clear sky,36.6177,-121.9166,Lovers Point Inn
2,519,Eureka,US,51.37,overcast clouds,40.8021,-124.1637,Eureka Inn Trademark By Wyndham
3,508,North Bend,US,50.92,overcast clouds,43.4065,-124.2243,Red Lion Hotel Coos Bay


In [36]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
vacation_itinerary = gmaps.directions_layer(
    start, end, waypoints=[stop1, stop2, stop3],
    travel_mode="DRIVING")
fig.add_layer(marker_layer)
fig.add_layer(vacation_itinerary)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")